In [1]:
"""
Fetch USGS discharge for Big Piney River, MO (site 06930000)
Date range: 2021-08-25 to 2021-08-26
"""

import os
import pandas as pd
import dataretrieval.nwis as nwis

# ------------------- CONFIG -------------------
SITE = "06930000"        # Big Piney River near Big Piney, MO
START = "2021-08-25"
END   = "2021-08-26"
PARAM = "00060"          # discharge (cubic feet per second)
OUTFILE = f"USGS_{SITE}_discharge_{START}_to_{END}.csv"
# ----------------------------------------------

print(f"Fetching discharge for site {SITE} ({START} → {END})")

# Try instantaneous first (iv), then fallback to daily (dv)
try:
    df_iv = nwis.get_record(sites=SITE, service="iv",
                            start=START, end=END, parameterCd=PARAM)
    if not df_iv.empty:
        df = df_iv.reset_index()
        df["discharge_cfs"] = df["00060"].astype(float)
        df["discharge_m3s"] = df["discharge_cfs"] * 0.0283168466
        df.to_csv(OUTFILE, index=False)
        print(f"✅ Saved instantaneous discharge to {OUTFILE}")
    else:
        raise ValueError("No instantaneous data returned; fetching daily.")
except Exception as e:
    print("Instantaneous fetch failed or unavailable:", e)
    df_dv, meta = nwis.get_record(sites=SITE, service="dv",
                                  start=START, end=END, parameterCd=PARAM)
    if not df_dv.empty:
        df = df_dv.reset_index()
        df["discharge_cfs"] = df["00060_Mean"].astype(float)
        df["discharge_m3s"] = df["discharge_cfs"] * 0.0283168466
        df.to_csv(OUTFILE, index=False)
        print(f"✅ Saved daily discharge to {OUTFILE}")
    else:
        print("❌ No discharge data found for this site/date range.")


Fetching discharge for site 06930000 (2021-08-25 → 2021-08-26)
✅ Saved instantaneous discharge to USGS_06930000_discharge_2021-08-25_to_2021-08-26.csv


In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# ==============================================================================
# PROJECT PARAMETERS (Confirmed from Data/Metadata/USGS)
# ==============================================================================

# --- 1. KOOTENAI RIVER PARAMETERS (Large River: Low Self-Cleansing Baseline) ---
K_Q = 198.40  # Mean Discharge (m^3/s)
K_M_INJ_KG = 14.514  # Injected Mass (kg)
K_L_M = 14659.0  # Distance to Sonde 1A (m)
K_T_INJ = '2017-09-27 04:10:40'  # Injection End Time (UTC)
K_STATION = '1A'

# --- 2. BIG PINEY RIVER PARAMETERS (Small River: High Self-Cleansing) ---
BP_Q = 5.777  # Mean Discharge (m^3/s) - CONFIRMED from USGS NWIS data
BP_M_INJ_KG = 2.40  # Injected Mass (kg)
BP_L_M = 1500.0  # Distance to Transect 1 (m)
BP_T_INJ = '2021-08-25 20:07:00'  # Injection Time (UTC)
BP_STATION = 1

# ==============================================================================
# CORE CALCULATION FUNCTION: Moment Method for Initial Assimilation Factor (Ka)
# ==============================================================================

def calculate_initial_ka(df, Q, M_INJ_KG, T_INJ_STR, L_M, river_name, time_col, conc_col, station_identifier, station_value):
    """Calculates the Initial Assimilation Factor (Ka) for a river reach."""
    df = df.copy()
    
    M_INJ_UG = M_INJ_KG * 1e9 # Convert kg to micrograms (ug)

    # 1. Time Normalization
    df[time_col] = pd.to_datetime(df[time_col])
    T_INJ = pd.to_datetime(T_INJ_STR)
    df['Time_s'] = (df[time_col] - T_INJ).dt.total_seconds()
    
    # 2. Filter Data 
    df_station = df[df[station_identifier] == station_value].copy()
    df_filtered = df_station[df_station['Time_s'] >= 0].sort_values(by='Time_s')
    
    times = df_filtered['Time_s'].values
    concentrations = df_filtered[conc_col].values # Conc units are ug/L or ppb (equivalent)

    # 3. Moment Calculation (M0 and t_bar)
    if len(times) < 5 or M_INJ_UG <= 0 or np.sum(concentrations) == 0:
        return {'River': river_name, 'Q (m^3/s)': Q, 'L (km)': L_M / 1000, 'M_Injected (kg)': M_INJ_KG, 'M_Passed (kg)': 0, 'M_Loss (%)': 100, 'Ka_initial (1/hr)': np.nan}

    M0 = np.trapezoid(concentrations, times) 
    M1 = np.trapezoid(concentrations * times, times)
    t_bar = M1 / M0 
    
    M_PASSED_UG = M0 * Q * 1000 

    # 4. Assimilation Factor (Ka) Calculation
    Ka_s = - (1 / t_bar) * np.log(M_PASSED_UG / M_INJ_UG)
    Ka_hr = Ka_s * 3600 # Convert to per hour

    return {
        'River': river_name,
        'Q (m^3/s)': Q,
        'L (km)': L_M / 1000,
        'M_Injected (kg)': M_INJ_KG,
        'M_Passed (kg)': M_PASSED_UG / 1e9,
        'M_Loss (%)': 100 * (1 - (M_PASSED_UG / M_INJ_UG)),
        'Ka_initial (1/hr)': Ka_hr
    }

# ==============================================================================
# EXECUTION AND EXPORT
# ==============================================================================

# --- 1. KOOTENAI RIVER EXECUTION ---
df_kootenai = pd.read_csv('KootenaiMooredSondeConcentrationData.csv')
kootenai_result = calculate_initial_ka(
    df_kootenai, K_Q, K_M_INJ_KG, K_T_INJ, K_L_M, 'Kootenai', 
    'UTCtimeStamp', 'DyeConc', 'Sonde', K_STATION
)

# --- 2. BIG PINEY RIVER EXECUTION ---
df_bp = pd.read_csv('BigPiney_RWTFluorometers.csv')
df_bp['UTCtimeStamp'] = pd.to_datetime(df_bp['date'] + ' ' + df_bp['time'])
bp_result = calculate_initial_ka(
    df_bp, BP_Q, BP_M_INJ_KG, BP_T_INJ, BP_L_M, 'Big Piney', 
    'UTCtimeStamp', 'rwt_ppb', 'transect_ID', BP_STATION
)

# --- 3. CREATE FINAL COMPARISON TABLE ---
df_final_comparison = pd.DataFrame([bp_result, kootenai_result])

# Clean up M_Loss percentage (ensure it doesn't show negative, set minimum to 0)
df_final_comparison['M_Loss (%)'] = df_final_comparison['M_Loss (%)'].apply(lambda x: max(0, x))


# --- 4. EXPORT TO CSV FILE ---
CSV_FILE_PATH = "Ka_Comparison_Results.csv"
df_final_comparison.round(4).to_csv(CSV_FILE_PATH, index=False)


# --- 5. PRINT SUMMARY ---
print("===============================================================")
print("  FINAL COMPARATIVE ASSIMILATION FACTOR (Ka) RESULTS")
print("  Successfully saved to Ka_Comparison_Results.csv")
print("===============================================================")

print("\n--- Key Findings ---")
print(f"Kootenai River Ka (Large River): {kootenai_result['Ka_initial (1/hr)']:.4f} /hr")
print(f"Big Piney River Ka (Small River): {bp_result['Ka_initial (1/hr)']:.4f} /hr")
print(f"\nConclusion: The Big Piney River shows a mass loss rate {bp_result['Ka_initial (1/hr)'] / kootenai_result['Ka_initial (1/hr)']:.0f} times higher than the Kootenai River.")

  FINAL COMPARATIVE ASSIMILATION FACTOR (Ka) RESULTS
  Successfully saved to Ka_Comparison_Results.csv

--- Key Findings ---
Kootenai River Ka (Large River): 0.0034 /hr
Big Piney River Ka (Small River): 0.0692 /hr

Conclusion: The Big Piney River shows a mass loss rate 20 times higher than the Kootenai River.
